In [5]:
from tile_classifier import TileClassifier
from export_model import export_model
import tensorflow as tf

In [6]:
if __name__ == '__main__':
    model_path ='models/tile_classifier.h5'
    data_set_path = './Scrabble_Detector/data/data_set_tile'
    model = TileClassifier(data_set_path,model_path)
    model.train_evaluate(epochs=5)
    class_indices = model.get_class_indices()
    print(class_indices)
    image_path = './data/testy/000.jpg'
    model.validate(image_path)
    
    export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax")

FileNotFoundError: [Errno 2] No such file or directory: './Scrabble_Detector/data/data_set_tile'